In [1]:
%matplotlib inline

In [15]:
import subprocess
import glob
from Bio import Phylo

In [29]:
def bash_command(cmd):
    return subprocess.Popen(cmd, shell=True, executable='/bin/bash')

# Re-write tree / fastas without periods in names

In [23]:
tree = Phylo.read('../Data/Tree_files/Order_Enterobacterales_Representative/RAxML_bestTree.RepresentativeSet101seqsFLAT', 'newick')
for node in tree.get_terminals():
    node.name = node.name.replace('.', '_')
Phylo.write(tree, './tree_no_periods.newick', 'newick');

# Run HYPHY on all

In [36]:
for infile_name in glob.glob('../Data/Orthologs/Order_Enterobacterales_Representative/mfastas_nt/*001.mfasta'):
    gene_name = infile_name.split('/')[-1].rstrip('.mfasta')
    print(infile_name)
    infile = open(infile_name, 'r').readlines()
    with open('temp.fa', 'w') as outfile:
        for line in infile:
            if line[0] == '>':
                outfile.write(line.replace('.', '_'))
            else:
                outfile.write(line)
    command = 'cat temp.fa tree_no_periods.newick > temp_hyphy_align.txt'
    process = bash_command(command)
    process.communicate()
    with open('./proteinER-master/src/temp_hyphy_setup.txt', 'w') as outfile:
        outfile.write('INFILE = /Users/adamhockenberry/Projects/2017/Internal_SD_conservation/Code/temp_hyphy_align.txt\n')
        outfile.write('OUTFILE = /Users/adamhockenberry/Projects/2017/Internal_SD_conservation/Results/Order_Enterobacterales_Representative/hyphy_jc/{}.rates'.format(gene_name))
    command = 'HYPHYMP CPU=2 /Users/adamhockenberry/Projects/2017/Internal_SD_conservation/Code/proteinER-master/src/fitrates_JC_equal_NT.bf <<< temp_hyphy_setup.txt'
    process = bash_command(command)
    process.communicate()

../Data/Orthologs/Order_Enterobacterales_Representative/mfastas_nt/VBIEscCol129921_0001.mfasta
../Data/Orthologs/Order_Enterobacterales_Representative/mfastas_nt/VBIEscCol129921_1001.mfasta
../Data/Orthologs/Order_Enterobacterales_Representative/mfastas_nt/VBIEscCol129921_2001.mfasta
../Data/Orthologs/Order_Enterobacterales_Representative/mfastas_nt/VBIEscCol129921_3001.mfasta
